In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
# from keras.layers.merge import Concatenate
from keras.layers import *
from keras.layers import TimeDistributed, Lambda
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence, text

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5490517620033081668, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3212502630
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13651258504071257096
 physical_device_desc: "device: 0, name: Quadro M1200, pci bus id: 0000:01:00.0, compute capability: 5.0"]

In [3]:
data = pd.read_csv('../data/quora-questions.csv')
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
y = data.is_duplicate.values
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
tk = text.Tokenizer(num_words=200000)

max_len = 40
tk.fit_on_texts(list(data.question1.values.astype(str)) + list(data.question2.values.astype(str)))

x1 = tk.texts_to_sequences(data.question1.values.astype(str))
x1 = sequence.pad_sequences(x1, maxlen=max_len)

x2 = tk.texts_to_sequences(data.question2.values.astype(str))
x2 = sequence.pad_sequences(x2, maxlen=max_len)
x1.shape

(404351, 40)

In [ ]:
x1[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    2,    3,    1, 1228,   57, 1228, 2582,
          7,  575,    8,  763,  383,    8,   35])

In [ ]:
x2[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    2,    3,    1, 1228,   57,
       1228, 2582,    7,  575,    8,  763,  383])

In [ ]:
word_index = tk.word_index

ytrain_enc = np_utils.to_categorical(y)

embeddings_index = {}
emb_size = 300

f = open('../glove.6B/glove.6B.%sd.txt'%emb_size, encoding='utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

400000it [00:39, 10255.70it/s]


In [ ]:
print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, emb_size))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95603/95603 [00:00<00:00, 368097.77it/s]


In [ ]:
max_features = 200000
filter_length = 5
nb_filter = 64
pool_length = 4

model = Sequential()
print('Build model...')

Build model...


In [ ]:
model1 = Sequential()
model1.add(Embedding(len(word_index) + 1,
                     emb_size,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))

model1.add(TimeDistributed(Dense(emb_size, activation='relu')))
model1.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(300,)))

model2 = Sequential()
model2.add(Embedding(len(word_index) + 1,
                     emb_size,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))

model2.add(TimeDistributed(Dense(emb_size, activation='relu')))
model2.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(300,)))

model3 = Sequential()
model3.add(Embedding(len(word_index) + 1,
                     emb_size,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))
model3.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))
model3.add(Dropout(0.2))

model3.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))

model3.add(GlobalMaxPooling1D())
model3.add(Dropout(0.2))

model3.add(Dense(emb_size))
model3.add(Dropout(0.2))
model3.add(BatchNormalization())

model4 = Sequential()
model4.add(Embedding(len(word_index) + 1,
                     emb_size,
                     weights=[embedding_matrix],
                     input_length=40,
                     trainable=False))
model4.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))
model4.add(Dropout(0.2))

model4.add(Convolution1D(nb_filter=nb_filter,
                         filter_length=filter_length,
                         border_mode='valid',
                         activation='relu',
                         subsample_length=1))

model4.add(GlobalMaxPooling1D())
model4.add(Dropout(0.2))

model4.add(Dense(emb_size))
model4.add(Dropout(0.2))
model4.add(BatchNormalization())

model5 = Sequential()
model5.add(Embedding(len(word_index) + 1, emb_size, input_length=40, dropout=0.2))
model5.add(LSTM(emb_size, dropout_W=0.2, dropout_U=0.2))

model6 = Sequential()
model6.add(Embedding(len(word_index) + 1, emb_size, input_length=40, dropout=0.2))
model6.add(LSTM(emb_size, dropout_W=0.2, dropout_U=0.2))

C:\Users\pbhavsa\AppData\Local\Continuum\anaconda3\envs\quora\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=5, strides=1, padding="valid")`
C:\Users\pbhavsa\AppData\Local\Continuum\anaconda3\envs\quora\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=5, strides=1, padding="valid")`
C:\Users\pbhavsa\AppData\Local\Continuum\anaconda3\envs\quora\lib\site-packages\ipykernel_launcher.py:57: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=5, strides=1, padding="valid")`
C:\Users\pbhavsa\AppData\Local\Continuum\anaconda3\envs\quora\lib\site-packages\ipykernel_launcher.py:64: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=5, strides=1, padding="valid")`
C:\Users

In [ ]:
submodel_inputs = [model.input for model in [model1, model2, model3, model4, model5, model6]]
submodel_outputs = [model.output for model in [model1, model2, model3, model4, model5, model6]]

mergedout = Concatenate(axis=1)(submodel_outputs)

mergedout = BatchNormalization()(mergedout)

mergedout = Dense(emb_size)(mergedout)
mergedout = PReLU()(mergedout)
mergedout = Dropout(0.2)(mergedout)
mergedout = BatchNormalization()(mergedout)

# mergedout = Dense(300)(mergedout)
# mergedout = PReLU()(mergedout)
# mergedout = Dropout(0.2)(mergedout)
# mergedout = BatchNormalization()(mergedout)

# mergedout = Dense(300)(mergedout)
# mergedout = PReLU()(mergedout)
# mergedout = Dropout(0.2)(mergedout)
# mergedout = BatchNormalization()(mergedout)

mergedout = Dense(1)(mergedout)
mergedout = Activation('sigmoid')(mergedout)

checkpoint = ModelCheckpoint('../data/quora-model.h5', monitor='val_acc', save_best_only=True, verbose=2)

In [ ]:
newModel = Model(submodel_inputs, mergedout)
newModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
newModel.fit(x=[x1, x2, x1, x2, x1, x2], y=y, batch_size=4, epochs=1, verbose=1, validation_split=0.1, shuffle=True, callbacks=[checkpoint])

Train on 363915 samples, validate on 40436 samples
Epoch 1/1
 23592/363915 [>.............................] - ETA: 7:38:08 - loss: 0.6564 - acc: 0.6381